<a href="https://colab.research.google.com/github/raymondkim777/JobListingDataAnalysis/blob/main/Data_Analysis_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Downloading & Installing Kaggle

In [ ]:
'''
!pip install kaggle
from google.colab import files
files.upload()
'''

'\n!pip install kaggle\nfrom google.colab import files\nfiles.upload()\n'

In [ ]:
'''
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/Work/Projects/Data_Analysis/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle'''

'\n!mkdir -p ~/.kaggle\n!cp /content/drive/MyDrive/Work/Projects/Data_Analysis/kaggle.json ~/.kaggle/\n!chmod 600 ~/.kaggle/kaggle.json\n!ls ~/.kaggle'

In [ ]:
# !kaggle datasets download -d shivamb/real-or-fake-fake-jobposting-prediction

In [ ]:
# !ls
# !unzip  real-or-fake-fake-jobposting-prediction.zip

#Data Manipulation

Creating DataFrame sampling 1400 job listings (true/false)

In [2]:
import pandas as pd

file = "./drive/MyDrive/Work/Projects/Data_Analysis/fake_job_postings.csv"
df = pd.read_csv(file)
# df.info()

# print(pd.isna(df))
df.fillna(0, inplace=True)
# print(pd.isna(df))
# df.corr()
# df.describe()
col = df.columns
col

Index(['job_id', 'title', 'location', 'department', 'salary_range',
       'company_profile', 'description', 'requirements', 'benefits',
       'telecommuting', 'has_company_logo', 'has_questions', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function',
       'fraudulent'],
      dtype='object')

In [3]:
df2 = df.loc[:, [col[1], col[5], col[6], col[7], col[17]]]
df2 = df2.drop(df2.loc[df2.loc[:, 'requirements'] == 0].index, axis='index')
df2 = df2.drop(df2.loc[df2.loc[:, 'requirements'] == 0].index, axis='index')
df2 = df2.loc[df2.loc[:, 'description'].str.strip().str.len() > 0]

col2 = df2.columns
df2 = df2.loc[:, [col2[0], col2[2], col2[3], col2[4]]]

df_t = df2.loc[df2.loc[:, col2[-1]] == 0]
df_f = df2.loc[df2.loc[:, col2[-1]] == 1]
df_t = df_t.sample(n=700, replace=False)
df_f = df_f.sample(n=700, replace=False)

sample_df = pd.concat([df_t, df_f], ignore_index=True)
sample_df

,title,description,requirements,fraudulent
0,"Office Manager, Accountant, Europe","Transifex, a Greek-born company internationall...",The following are requirements (must-haves). I...,0
1,FX Institutional Sales / Business Development ...,Responsibilities:Identify potential new client...,Good command of English and the language of th...,0
2,Security Officer (SO),SMGI Employment OpportunitiesSecurity Manageme...,What are we looking for? SMGI employs highly t...,0
3,Child Care Worker,"Committed and enthusiastic, you'll work as par...",Strong IT skills.Previous care working experie...,0
4,Big Data / Hadoop Engineer,We are gathering all of the world's data relat...,"Deep knowledge of data mining, machine learnin...",0
...,...,...,...,...
1395,Customer Service Rep Call Center,"BACKGROUND/EXPERIENCE desired:* Warm, positive...",EDUCATIONThe highest level of education desire...,1
1396,Data Entry,We are seeking extremely motivated and experie...,Have a yahoo messenger which you would be wor...,1
1397,Brand & Logo Design Contest,"Calling all hungry, young &amp; fresh designer...",We need your CREATIVITY for a full Brand Desig...,1
1398,Payroll Data Coordinator Positions - Earn $100...,We are a full-service marketing and staffing f...,RequirementsAll you need is access to the Inte...,1


# LSTM (Long Short Term Memory) - RNN Subtype

RNN (recurrent neural network) is one of the two most commonly used neural networks, with the other being CNN (convolutional neural network). Due to the nature of both, CNN is often used for processing image data and RNN for sequential data. As text data is sequential, we aim to use RNN to analyze and categorize the texts in 'description' into 0 or 1 for 'fraudulent'.

However, when processing data, RNN is only able to access adjacent data, and not data further away or processed before. Thus, while it is theoretically possible for RNN to learn long-term dependencies by backpropogating through time, in practice RNN struggles to handle them effectively due to the vanishing gradient problem. Thus an altered version of RNN will be used, known as LSTM (Long Short Term Memory), that is able to adequately perform backpropagation.

LSTM utilizes structures called memory cells to store information. The memory cells regulate the storage, updating, and deletion of information through various gates (input, forget, output), enabling LSTM to forget unncessary information and only retain what is needed. Thus LSTM demonstrates far better performance than RNN in fields such as NLP, speech recognition, and time series forecasting, where contextual information is crucial.

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout


# 데이터 전처리
X = sample_df['title'] + ' ' + sample_df['description'] + ' ' + sample_df['requirements']
y = sample_df['fraudulent']

# 훈련 및 테스트 세트 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 텍스트 토큰화 및 시퀀스 패딩
max_words = 5000
max_len = 200
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# LSTM 모델 정의
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 훈련
model.fit(X_train_pad, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
28/28 [==============================] - 12s 336ms/step - loss: 0.6864 - accuracy: 0.5525 - val_loss: 0.6558 - val_accuracy: 0.7768
Epoch 2/10
28/28 [==============================] - 8s 272ms/step - loss: 0.4764 - accuracy: 0.8225 - val_loss: 0.3659 - val_accuracy: 0.8527
Epoch 3/10
28/28 [==============================] - 9s 335ms/step - loss: 0.2033 - accuracy: 0.9353 - val_loss: 0.3831 - val_accuracy: 0.8393
Epoch 4/10
28/28 [==============================] - 8s 282ms/step - loss: 0.0970 - accuracy: 0.9710 - val_loss: 0.4468 - val_accuracy: 0.7902
Epoch 5/10
28/28 [==============================] - 9s 315ms/step - loss: 0.0799 - accuracy: 0.9821 - val_loss: 0.4937 - val_accuracy: 0.8036
Epoch 6/10
28/28 [==============================] - 9s 334ms/step - loss: 0.0275 - accuracy: 0.9967 - val_loss: 0.4959 - val_accuracy: 0.8482
Epoch 7/10
28/28 [==============================] - 8s 281ms/step - loss: 0.0117 - accuracy: 0.9978 - val_loss: 0.5349 - val_accuracy: 0.8170
Epoch